# Previsão de Demanda (Restaurante)


In [2]:
# 0. Setup: imports e configurações
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error
import joblib
import warnings
warnings.filterwarnings('ignore')


# métricas
def smape(y_true, y_pred):
    y_true = np.array(y_true)
    y_pred = np.array(y_pred)
    denom = (np.abs(y_true) + np.abs(y_pred)) / 2
    mask = denom == 0
    denom[mask] = 1.0
    return 100/len(y_true) * np.sum(np.abs(y_pred - y_true) / denom)


In [3]:

path = '../data/raw/comensais22-24-clean.csv'
df = pd.read_csv(path, parse_dates=['Data'], index_col='Data')
df.sort_index(inplace=True)

df.rename(columns=lambda c: c.strip().replace(' ', '_').replace('Comensais_', ''), inplace=True)
df.columns = [c.lower().replace('ã','a').replace('á','a').replace('é','e').replace('í','i').replace('ó','o').replace('ú','u') for c in df.columns]

print('Colunas:', df.columns.tolist())
df.head()


Colunas: ['almoço', 'janta']


,almoço,janta
Data,,
2022-03-16,315.0,100.0
2022-03-17,220.0,110.0
2022-03-18,350.0,120.0
2022-03-19,NaN,NaN
2022-03-20,NaN,NaN


In [4]:
# 2. Tratar ausências e padronizar frequência
full_idx = pd.date_range(df.index.min(), df.index.max(), freq='D')
df = df.reindex(full_idx)
df.index.name = 'date'

if 'almoco' not in df.columns and 'comensais_almoço' in df.columns:
    df['almoco'] = df['comensais_almoço']
if 'janta' not in df.columns and 'comensais_janta' in df.columns:
    df['janta'] = df['comensais_janta']

cols = [c for c in df.columns if 'almoco' in c or 'janta' in c]
print('Séries encontradas:', cols)

for c in cols:
    df[c+'_is_observed'] = (~df[c].isna()).astype(int)

df[cols] = df[cols].fillna(method='ffill').fillna(df[cols].rolling(window=7, min_periods=1).mean())

df[cols].isna().sum(), df.head()


Séries encontradas: ['janta']


(janta    0
 dtype: int64,
             almoço  janta  janta_is_observed
 date                                        
 2022-03-16   315.0  100.0                  1
 2022-03-17   220.0  110.0                  1
 2022-03-18   350.0  120.0                  1
 2022-03-19     NaN  120.0                  0
 2022-03-20     NaN  120.0                  0)

In [5]:
df = df.rename(columns={
    'Comensais_almoço': 'almoco',
    'Comensais_janta': 'janta',
    'almoço': 'almoco',
    'janta': 'janta',
    'Almoço': 'almoco',
    'Janta': 'janta',
    'almoco_is_observed': 'almoco_is_observed',
    'almoço_is_observed': 'almoco_is_observed',
    'janta_is_observed': 'janta_is_observed'
})

print(df.columns)
df["almoco"] = df["almoco"].fillna(0)
df["janta"] = df["janta"].fillna(0)


if 'almoco_is_observed' not in df.columns:
    df['almoco_is_observed'] = (~df['almoco'].isna()).astype(int)

if 'janta_is_observed' not in df.columns:
    df['janta_is_observed'] = (~df['janta'].isna()).astype(int)


def make_features(df, target_col, lags=[1,7,14], rolls=[7,14]):
    X = pd.DataFrame(index=df.index)
    
    X['dayofweek'] = df.index.dayofweek
    X['month'] = df.index.month
    X['day'] = df.index.day
    X['is_weekend'] = (X['dayofweek'] >= 5).astype(int)

    for l in lags:
        X[f'lag_{l}'] = df[target_col].shift(l)

    for r in rolls:
        X[f'roll_mean_{r}'] = df[target_col].shift(1).rolling(window=r, min_periods=1).mean()

    X['diff_1_7'] = X['lag_1'] - X['lag_7']
    
    return X


target_l = 'almoco'
target_j = 'janta'

X_l = make_features(df, target_l)
X_j = make_features(df, target_j)


try:
    import holidays
    br_holidays = holidays.Brazil()
    df['is_holiday'] = df.index.map(lambda d: 1 if d in br_holidays else 0)
except:
    df['is_holiday'] = 0

data_l = pd.concat([
    df[[target_l]],
    X_l,
    df[['almoco_is_observed', 'is_holiday']]
], axis=1)

data_j = pd.concat([
    df[[target_j]],
    X_j,
    df[['janta_is_observed', 'is_holiday']]
], axis=1)

data_l = data_l.dropna()
data_j = data_j.dropna()

print("Features almoço:")
display(data_l.head())

print("Features janta:")
display(data_j.head())


Index(['almoco', 'janta', 'janta_is_observed'], dtype='object')
Features almoço:


,almoco,dayofweek,month,day,is_weekend,lag_1,lag_7,lag_14,roll_mean_7,roll_mean_14,diff_1_7,almoco_is_observed,is_holiday
date,,,,,,,,,,,,,
2022-03-30,350.0,2,3,30,0,400.0,440.0,315.0,220.000000,225.357143,-40.0,1,0
2022-03-31,350.0,3,3,31,0,350.0,350.0,220.0,207.142857,227.857143,0.0,1,0
2022-04-01,300.0,4,4,1,0,350.0,0.0,350.0,207.142857,237.142857,350.0,1,0
2022-04-02,0.0,5,4,2,1,300.0,0.0,0.0,250.000000,233.571429,300.0,1,0
2022-04-03,0.0,6,4,3,1,0.0,0.0,0.0,250.000000,233.571429,0.0,1,0


Features janta:


,janta,dayofweek,month,day,is_weekend,lag_1,lag_7,lag_14,roll_mean_7,roll_mean_14,diff_1_7,janta_is_observed,is_holiday
date,,,,,,,,,,,,,
2022-03-30,260.0,2,3,30,0,290.0,290.0,100.0,255.714286,201.428571,0.0,1,0
2022-03-31,260.0,3,3,31,0,260.0,240.0,110.0,251.428571,212.857143,20.0,1,0
2022-04-01,190.0,4,4,1,0,260.0,240.0,120.0,254.285714,223.571429,20.0,1,0
2022-04-02,190.0,5,4,2,1,190.0,240.0,120.0,247.142857,228.571429,-50.0,0,0
2022-04-03,190.0,6,4,3,1,190.0,240.0,120.0,240.000000,233.571429,-50.0,0,0


In [6]:
# 4. Backtesting (Walk-forward) para um modelo sklearn
def walk_forward_forecast(data, target_col, model_cls, model_kwargs=None, initial_train_days=90, fh=7, step=7):
    '''
    data: dataframe com target_col e features (features start after target_col)
    model_cls: sklearn-like estimator class
    initial_train_days: tamanho da janela inicial de treino (dias)
    fh: horizon de previsão (dias)
    step: passo de avanço (dias)
    '''
    model_kwargs = model_kwargs or {}
    preds = []
    trues = []
    dates = []
    n = data.shape[0]
    start_idx = 0
    while True:
        train_end = start_idx + initial_train_days
        if train_end + fh > n:
            break
        train = data.iloc[start_idx:train_end]
        test = data.iloc[train_end:train_end+fh]
        X_train = train.drop(columns=[target_col])
        y_train = train[target_col]
        X_test = test.drop(columns=[target_col])
        y_test = test[target_col]
        model = model_cls(**model_kwargs)
        model.fit(X_train, y_train)
        y_pred = model.predict(X_test)
        preds.extend(y_pred.tolist())
        trues.extend(y_test.tolist())
        dates.extend(test.index.tolist())
        start_idx += step
    return pd.DataFrame({'date':dates, 'y_true':trues, 'y_pred':preds}).set_index('date')

# Testar com RandomForest para almoço
features_l = data_l.drop(columns=[target_l])
bt_l = walk_forward_forecast(data_l, target_l, RandomForestRegressor, model_kwargs={'n_estimators':100,'random_state':42},
                             initial_train_days=90, fh=7, step=7)
print('Backtest RandomForest — almoço')
print('SMAPE:', smape(bt_l['y_true'], bt_l['y_pred']))
print('MAE:', mean_absolute_error(bt_l['y_true'], bt_l['y_pred']))


Backtest RandomForest — almoço
SMAPE: 62.33572910666682
MAE: 50.817578692493946


In [7]:
# 5. Modelo LLM — template e função de chamada

def build_llm_prompt(window_df, horizon=7, context_text=None):
    '''
    window_df: pandas dataframe com índice datetime e colunas almoco/janta
    horizon: dias a prever
    context_text: string com eventos/feriados/promoções
    '''
    prompt = 'Você é um modelo que prevê número de clientes por dia para um restaurante.\n'
    if context_text:
        prompt += f'Contexto: {context_text}\n'
    prompt += f'Histórico recente (data, almoco, janta) — mostre previsões para os próximos {horizon} dias:\n'

    for d, row in window_df.iterrows():
        alm = row.get("almoco", 0)
        jan = row.get("janta", 0)

        if pd.isna(alm):
            alm = 0
        if pd.isna(jan):
            jan = 0

        prompt += f'{d.date()}: almoco={int(alm)}, janta={int(jan)}\n'

    prompt += (
        f'\nRetorne uma lista JSON com {{'
        '"date": "YYYY-MM-DD", "almoco": <int>, "janta": <int>'
        f'}} para os próximos {horizon} dias.'
    )

    return prompt


def call_openai_llm(prompt, api_key=None, model='gpt-4o-mini'):
    try:
        import openai
    except Exception as e:
        raise ImportError('biblioteca openai não instalada. Instale com `pip install openai` para usar LLM.') from e
    
    if api_key is None:
        api_key = os.environ.get('OPENAI_API_KEY')
    if api_key is None:
        raise ValueError('Chave de API não fornecida.')
    
    openai.api_key = api_key

    resp = openai.chat.completions.create(
        model=model,
        messages=[{"role": "user", "content": prompt}],
        max_tokens=500,
        temperature=0.0
    )
    return resp

last_window = df.iloc[-30:].copy()  
example_prompt = build_llm_prompt(last_window, horizon=7, context_text='feriado municipal em 2024-08-15')

print('Exemplo de prompt LLM (preview):')


print(example_prompt[:1000])


Exemplo de prompt LLM (preview):
Você é um modelo que prevê número de clientes por dia para um restaurante.
Contexto: feriado municipal em 2024-08-15
Histórico recente (data, almoco, janta) — mostre previsões para os próximos 7 dias:
2024-09-05: almoco=380, janta=305
2024-09-06: almoco=375, janta=220
2024-09-07: almoco=0, janta=220
2024-09-08: almoco=0, janta=220
2024-09-09: almoco=375, janta=280
2024-09-10: almoco=345, janta=285
2024-09-11: almoco=395, janta=290
2024-09-12: almoco=380, janta=270
2024-09-13: almoco=365, janta=220
2024-09-14: almoco=0, janta=220
2024-09-15: almoco=0, janta=220
2024-09-16: almoco=340, janta=260
2024-09-17: almoco=390, janta=310
2024-09-18: almoco=420, janta=285
2024-09-19: almoco=380, janta=265
2024-09-20: almoco=350, janta=200
2024-09-21: almoco=0, janta=200
2024-09-22: almoco=0, janta=200
2024-09-23: almoco=355, janta=260
2024-09-24: almoco=360, janta=260
2024-09-25: almoco=390, janta=250
2024-09-26: almoco=360, janta=250
2024-09-27: almoco=280, janta=

### 5.1 Como executar backtesting com LLM 
O LLM é mais caro/mais lento que um modelo sklearn. Recomenda-se:
- Fazer predição em janelas (janela deslizante).  
- Usar temperatura 0.0 para previsões determinísticas.  
- Limitar chamadas durante backtesting (ex.: usar amostragem reduzida para validação inicial).  
- Parsear a resposta JSON retornada pelo LLM.



In [8]:
# 6. Comparação e previsão final

final_rf = RandomForestRegressor(n_estimators=200, random_state=42)
X_full_l = data_l.drop(columns=[target_l])
y_full_l = data_l[target_l]
final_rf.fit(X_full_l, y_full_l)

horizon = 30
last_row = df.iloc[-1:].copy()
preds = []
idxs = []
current_df = df.copy()

for i in range(horizon):
    X_next = make_features(current_df, 'almoco').iloc[-1:].copy()
    X_next['is_holiday'] = 1 if X_next.index[0].date() in [] else 0
    
    X_next = X_next.reindex(columns=X_full_l.columns).fillna(0)
    
    pred = final_rf.predict(X_next)[0]
    preds.append(pred)

    next_date = current_df.index[-1] + pd.Timedelta(days=1)
    idxs.append(next_date)

    new_row = pd.Series({'almoco': pred, 'janta': np.nan}, name=next_date)

    current_df = pd.concat([current_df, new_row.to_frame().T])

df_forecast_rf = pd.DataFrame(
    {'date': idxs, 'almoco_pred_rf': preds}
).set_index('date')

out_path = '../data/processed/forecast_almoco_rf_30d.csv'
df_forecast_rf.to_csv(out_path)
print('Previsão RF salva em', out_path)



Previsão RF salva em ../data/processed/forecast_almoco_rf_30d.csv


In [91]:
# 6.1 Comparação e previsão final

final_rf = RandomForestRegressor(n_estimators=200, random_state=42)
X_full_l = data_l.drop(columns=[target_l])
y_full_l = data_l[target_l]
final_rf.fit(X_full_l, y_full_l)

horizon = 30
last_row = df.iloc[-1:].copy()
preds = []
idxs = []
current_df = df.copy()

for i in range(horizon):
    X_next = make_features(current_df, 'janta').iloc[-1:].copy()
    X_next['is_holiday'] = 1 if X_next.index[0].date() in [] else 0
    
    X_next = X_next.reindex(columns=X_full_l.columns).fillna(0)
    
    pred = final_rf.predict(X_next)[0]
    preds.append(pred)

    next_date = current_df.index[-1] + pd.Timedelta(days=1)
    idxs.append(next_date)

    new_row = pd.Series({'janta': pred, 'almoco': np.nan}, name=next_date)

    current_df = pd.concat([current_df, new_row.to_frame().T])

df_forecast_rf = pd.DataFrame(
    {'date': idxs, 'janta_pred_rf': preds}
).set_index('date')

out_path = '../data/processed/forecast_janta_rf_30d.csv'
df_forecast_rf.to_csv(out_path)
print('Previsão RF salva em', out_path)


Previsão RF salva em ../data/processed/forecast_janta_rf_30d.csv


In [92]:
# 6.2 Comparação e previsão final

# Treinar RF final para janta

final_rf = RandomForestRegressor(n_estimators=200, random_state=42)
X_full_l = data_l.drop(columns=[target_l])
y_full_l = data_l[target_l]
final_rf.fit(X_full_l, y_full_l)

horizon = 15
last_row = df.iloc[-1:].copy()
preds = []
idxs = []
current_df = df.copy()

for i in range(horizon):
    X_next = make_features(current_df, 'janta').iloc[-1:].copy()
    X_next['is_holiday'] = 1 if X_next.index[0].date() in [] else 0
    
    X_next = X_next.reindex(columns=X_full_l.columns).fillna(0)
    
    pred = final_rf.predict(X_next)[0]
    preds.append(pred)

    next_date = current_df.index[-1] + pd.Timedelta(days=1)
    idxs.append(next_date)

    new_row = pd.Series({'janta': pred, 'almoco': np.nan}, name=next_date)

    current_df = pd.concat([current_df, new_row.to_frame().T])

df_forecast_rf = pd.DataFrame(
    {'date': idxs, 'janta_pred_rf': preds}
).set_index('date')

out_path = '../data/processed/forecast_janta_rf_15d.csv'
df_forecast_rf.to_csv(out_path)
print('Previsão RF salva em', out_path)



Previsão RF salva em ../data/processed/forecast_janta_rf_15d.csv


In [9]:
def build_llm_prompt_v2(window_df, horizon=7, domain_name="demand forecasting", context_text=None):
    """
    """
    prompt = ""

    prompt += (
        f"You are a forecasting model specialized in {domain_name}. "
        f"Your task is to predict the number of restaurant customers for future dates.\n"
    )

    if context_text:
        prompt += f"Additional context: {context_text}\n"

    prompt += (
        "\nInstructions:\n"
        f"- Use only the historical values provided below.\n"
        f"- Capture trend, weekly seasonality, and short-term fluctuations.\n"
        f"- Avoid guessing special events unless explicitly mentioned.\n"
        f"- Generate forecasts for the next {horizon} days.\n"
    )

    prompt += "\nHistorical data (date, almoco, janta):\n"

    for d, row in window_df.iterrows():
        alm = row.get("almoco", 0)
        jan = row.get("janta", 0)
        alm = 0 if pd.isna(alm) else int(alm)
        jan = 0 if pd.isna(jan) else int(jan)
        prompt += f"{d.date()}: almoco={alm}, janta={jan}\n"

    prompt += (
        f"\nReturn ONLY a JSON list with this structure:\n"
        f'[\n'
        f'  {{"date": "YYYY-MM-DD", "almoco": <int>, "janta": <int>}},\n'
        f'  ... {horizon} items ...\n'
        f']\n'
        f"No explanations. No text outside JSON."
    )
    return prompt

prompt = build_llm_prompt_v2(window_df=df.iloc[-30:], horizon=7)
print(prompt)



You are a forecasting model specialized in demand forecasting. Your task is to predict the number of restaurant customers for future dates.

Instructions:
- Use only the historical values provided below.
- Capture trend, weekly seasonality, and short-term fluctuations.
- Avoid guessing special events unless explicitly mentioned.
- Generate forecasts for the next 7 days.

Historical data (date, almoco, janta):
2024-09-05: almoco=380, janta=305
2024-09-06: almoco=375, janta=220
2024-09-07: almoco=0, janta=220
2024-09-08: almoco=0, janta=220
2024-09-09: almoco=375, janta=280
2024-09-10: almoco=345, janta=285
2024-09-11: almoco=395, janta=290
2024-09-12: almoco=380, janta=270
2024-09-13: almoco=365, janta=220
2024-09-14: almoco=0, janta=220
2024-09-15: almoco=0, janta=220
2024-09-16: almoco=340, janta=260
2024-09-17: almoco=390, janta=310
2024-09-18: almoco=420, janta=285
2024-09-19: almoco=380, janta=265
2024-09-20: almoco=350, janta=200
2024-09-21: almoco=0, janta=200
2024-09-22: almoco=

In [12]:
from dotenv import load_dotenv
import os
import json
import re
import pandas as pd

def extract_json(text):
    try:
        json_str = re.search(r"\[.*\]", text, re.DOTALL).group(0)
        return json.loads(json_str)
    except Exception:
        print("ERRO ao extrair JSON da resposta:")
        print(text)
        return None


load_dotenv() 

def llm_walk_forward(
    df,
    horizon=7,
    window_size=30,
    step=7,          
    max_loops=None,     
    model="gpt-4o-mini",
    api_key=None
):

    results = []

    if api_key is None:
        api_key = os.environ.get("OPENAI_API_KEY")

    if api_key is None:
        raise ValueError("Nenhuma API KEY fornecida.")

    iter_range = list(range(0, len(df) - window_size - horizon, step))

    if max_loops is not None:
        iter_range = iter_range[:max_loops]

    for i in iter_range:

        window = df.iloc[i:i+window_size]
        true_future = df.iloc[i+window_size:i+window_size+horizon][["almoco", "janta"]]

        prompt = build_llm_prompt_v2(window, horizon=horizon)

        resp = call_openai_llm(prompt, api_key=api_key, model=model)

        raw_text = resp.choices[0].message.content
        print("\n===== RAW LLM RESPONSE =====")
        print(raw_text)

        pred_json = extract_json(raw_text)

        if pred_json is None:
            print(f"\nPULANDO JANELA {i}: resposta inválida\n")
            continue

        preds = pd.DataFrame(pred_json).set_index("date")
        preds.index = pd.to_datetime(preds.index)

        results.append((true_future, preds))

    return results

results = llm_walk_forward(
    df,
    horizon=7,
    window_size=30,
    step=7,
    max_loops=10,   
    model="gpt-4o-mini",
    api_key=None
)

print("Total de janelas geradas:", len(results))





===== RAW LLM RESPONSE =====
[
  {"date": "2022-04-15", "almoco": 0, "janta": 190},
  {"date": "2022-04-16", "almoco": 0, "janta": 190},
  {"date": "2022-04-17", "almoco": 350, "janta": 250},
  {"date": "2022-04-18", "almoco": 400, "janta": 200},
  {"date": "2022-04-19", "almoco": 330, "janta": 260},
  {"date": "2022-04-20", "almoco": 440, "janta": 290},
  {"date": "2022-04-21", "almoco": 350, "janta": 240}
]

===== RAW LLM RESPONSE =====
[
  {"date": "2022-04-22", "almoco": 0, "janta": 215},
  {"date": "2022-04-23", "almoco": 0, "janta": 215},
  {"date": "2022-04-24", "almoco": 340, "janta": 270},
  {"date": "2022-04-25", "almoco": 400, "janta": 270},
  {"date": "2022-04-26", "almoco": 300, "janta": 215},
  {"date": "2022-04-27", "almoco": 0, "janta": 215},
  {"date": "2022-04-28", "almoco": 0, "janta": 215}
]

===== RAW LLM RESPONSE =====
[
  {"date": "2022-04-29", "almoco": 350, "janta": 250},
  {"date": "2022-04-30", "almoco": 0, "janta": 250},
  {"date": "2022-05-01", "almoco": 0

In [13]:
def evaluate_llm_results(results):
    smapes = []
    maes = []
    valid_count = 0
    invalid_count = 0

    for true_df, pred_df in results:

        if true_df is None or pred_df is None:
            invalid_count += 1
            continue

        if len(true_df) != len(pred_df):
            invalid_count += 1
            continue

        true = true_df["almoco"].values.astype(float)
        pred = pred_df["almoco"].values.astype(float)

        if np.isnan(true).any() or np.isnan(pred).any():
            invalid_count += 1
            continue

        smapes.append(smape(true, pred))
        maes.append(mean_absolute_error(true, pred))
        valid_count += 1

    print(f"Janelas válidas: {valid_count}")
    print(f"Janelas descartadas: {invalid_count}")

    if len(smapes) == 0:
        return None, None

    return np.mean(smapes), np.mean(maes)


mean_smape, mean_mae = evaluate_llm_results(results) 
print("SMAPE médio do LLM:", mean_smape) 
print("MAE médio do LLM:", mean_mae)

Janelas válidas: 10
Janelas descartadas: 0
SMAPE médio do LLM: 43.462276744313144
MAE médio do LLM: 93.81428571428572
